Concept to know before start
- Registers (fastest)
- L1 cache (~32 KB)
- L2 cache
- RAM (slow)

Also:
- If working data fits inside L1 cache → FAST.
- If not → cache misses → SLOW.

In [1]:
# No cache reuse  - Baseline implementation
import numpy as np
import time
from numba import njit

@njit
def no_blocking(array_data, nruns):
    n = len(array_data)

    i = 0
    while i < nruns:
        j = 0
        while j < n:
            array_data[j] = 2.3 * array_data[j] + 1.2
            j += 1
        i += 1


In [2]:
# Cache blocked implementation
@njit
def cache_blocking(array_data, nruns, l1size):
    n = len(array_data)

    blockstart = 0

    while blockstart < n:

        # operate many times on SAME BLOCK
        b = 0
        while b < nruns:

            j = 0
            while j < l1size and (blockstart + j) < n:
                idx = blockstart + j
                array_data[idx] = 2.3 * array_data[idx] + 1.2
                j += 1

            b += 1

        blockstart += l1size

In [9]:
# Measuring the performance
N = 5000000
NRUNS = 2000

A1 = np.random.rand(N)
A2 = A1.copy()

# compile first
no_blocking(A1, 1)
cache_blocking(A2, 1, 1024)

start = time.time()
no_blocking(A1, NRUNS)
print("No blocking:", time.time()-start)

start = time.time()
cache_blocking(A2, NRUNS, 1024)
print("Cache blocking:", time.time()-start)


No blocking: 7.8998730182647705
Cache blocking: 0.9585721492767334


In [10]:
# Now we simulate increasing working-set size
sizes = [128, 256, 512, 1024, 2048, 4096, 8192, 16384]
ops = N * NRUNS
for l1 in sizes:
    A2 = A1.copy()

    start = time.time()
    cache_blocking(A2, NRUNS, l1)
    t = time.time() - start

    time_per_op = t / ops

    print(f"l1size={l1:6d}  time={t:.10f}  time/op={time_per_op:.2e}")

l1size=   128  time=0.8443634510  time/op=8.44e-11
l1size=   256  time=0.8876249790  time/op=8.88e-11
l1size=   512  time=0.8577971458  time/op=8.58e-11
l1size=  1024  time=0.8002614975  time/op=8.00e-11
l1size=  2048  time=0.8402786255  time/op=8.40e-11
l1size=  4096  time=0.8303656578  time/op=8.30e-11
l1size=  8192  time=1.2071485519  time/op=1.21e-10
l1size= 16384  time=1.3940382004  time/op=1.39e-10


*Note:*
Making the l1size parameter bigger means the computer operates on more data.  Operation time remains roughly constant for block sizes of about 4096 elements since the working set fits into the 32 KB L1 cache. When the block size exceeds this limit, time per operation increases a lot, suggesting that data no longer fits in L1 cache, and must go to the lower levels of cache.

By confirming the cache hierarchy model, this behavior also shows that cache blocking enhances performance, as it ensures reused data remains in the fastest cache level. 